In [1]:
import numpy as np
from PIL import Image

In [2]:
def writePGM(file, w, h, depth, flatImg):
    file.write('P5\n'.encode())
    file.write('{} {}\n'.format(w, h).encode())
    file.write('{}\n'.format(depth).encode())
    file.write(bytearray(flatImg))
    file.close()

In [3]:
def flatten(arr):
    return sum(arr, [])

In [4]:
def convert_to_gray(img):
    m,n = len(img[0]),len(img)
    result = [[255]*m for _ in range(n)]
    for i in range(n):
        for j in range(m):
            result[i][j] = int(sum(img[i][j])/3)
    return result

In [5]:
def segment(img,Tmin,Tmax):
    m = len(img[0])
    n = len(img)
    bg = [[255]*m for _ in range(n)]
    count = 0
    for i in range(n):
        for j in range(m):
            if Tmin <= img[i][j] and img[i][j] <= Tmax :
                bg[i][j] = 255
                count += 1
            else :
                bg[i][j] = 0
    return bg , count

In [6]:
def padding_center(img, pad):
    pading = [255]*(pad//2)
    l = len(img[0])
    for row in img :
        for zero in pading :
            row.insert(0, zero)
        row.extend(pading)
    pading = [[255]*(l + 2 * (pad//2)) for _ in range(pad//2)]
    for row in pading:
        img.insert(0, row)
    img.extend(pading)

In [7]:
def convolution(img,kernel):
    n = len(img)
    m = len(img[0])
    result_img = [[255]*m for _ in range(n)]
    for i in range(1,n-1):
        for j in range(1,m-1):
            temp = 0
            temp += img[i][j] * kernel[1][1]
            temp += img[i-1][j-1] * kernel[0][0]
            temp += img[i-1][j] * kernel[0][1]
            temp += img[i+1][j+1] * kernel[2][2]
            temp += img[i][j-1] * kernel[1][0]
            temp += img[i][j+1] * kernel[1][2]
            temp += img[i+1][j-1] * kernel[2][0]
            temp += img[i+1][j] * kernel[2][1]
            temp += img[i+1][j+1] * kernel[2][2]
            result_img[i][j] = int(temp)
    return result_img

In [8]:
def contrast_s(li):
    result = []
    c = np.amin(li)
    d = np.amax(li)
    a = 0
    b = 255
    for i in range(len(li)):
        row = []
        for j in range(len(li[0])):
            p = li[i][j]
            p_out = (p - c)*((b - a)/(d - c)) + a
            row.append(int(p_out))
        result.append(row)
    return result

In [9]:
img_1H = Image.open(".\WormHole_1H.tif")
# img_1H.show()
img_2H = Image.open(".\WormHole_2H.tif")
# img_1H.show()

In [10]:
img_np_1H = np.array(img_1H)
img_np_2H = np.array(img_2H)

In [11]:
gray_img_1H = convert_to_gray(img_np_1H)
gray_img_2H = convert_to_gray(img_np_2H)

In [12]:
save_dir = open("./result_1H.pgm","wb")
writePGM(save_dir , len(gray_img_1H[0]) , len(gray_img_1H) , 255 , flatten(gray_img_1H))

In [13]:
save_dir = open("./result_2H.pgm","wb")
writePGM(save_dir , len(gray_img_2H[0]) , len(gray_img_2H) , 255 , flatten(gray_img_2H))

In [83]:
pic , cc = segment(gray_img_1H,75,255)

In [14]:
new_gray_img_1H = gray_img_1H.copy()
new_gray_img_2H = gray_img_2H.copy()

In [15]:
kernel = np.array([[-1, -1, -1],[-1,  8, -1],[-1, -1, -1]])

In [16]:
padding_center(new_gray_img_1H,2)
padding_center(new_gray_img_2H,2)

In [17]:
raw_img_1H = convolution(new_gray_img_1H,kernel)
raw_img_2H = convolution(new_gray_img_2H,kernel)

In [18]:
con_img_1H = contrast_s(raw_img_1H)
con_img_2H = contrast_s(raw_img_2H)

In [19]:
save_dir = open("./edge_1H.pgm","wb")
writePGM(save_dir , len(con_img_1H[0]) , len(con_img_1H) , 255 , flatten(con_img_1H))

In [20]:
save_dir = open("./edge_2H.pgm","wb")
writePGM(save_dir , len(con_img_2H[0]) , len(con_img_2H) , 255 , flatten(con_img_2H))